# Currency risks for Swiss residents


## CMD: A Short Summary

If one wants to easily replicate the main findings of this paper, they might execute the command lines below (from the root folder). Later, we'll explore each part in further detail and explore the more nuanced analysis allowed by our implementations.

In [ ]:
:: Setup
setup.bat

:: Data retrieval and processing
python src/data/fred_data_retrieval.py
python src/data/oecd_data_retrieval.py
python src/data/fred_oecd_data_concatenation.py

:: Analysis, modeling and reporting
Rscript src/models/CODE_FAMA_CURRENCY.R
python src/analysis/performances.py

:: LaTeX files compiling

## Data retrieval

We use FRED to gather ForEx and VIX monthly data through FRED's API. We additionally gather German and Italian 10y-bond yields.

We obtain short-term interest rates through an html request to the OECD database.

CAUTION: OECD data for Japan (JPY) is available only since April 2002. Thus, it's not recommended to use a start previous to this one.

### FRED

In [24]:
import pandas as pd
import src.data.fred_data_retrieval as fred

# Define a time horizon
start_date = pd.Timestamp('2002-04-01')
end_date = pd.Timestamp('2023-10-31')

# Enter FRED API key
api_key = '41641234dac871704cd09b2b8dd163ac'

# Call to download function
fred.download(start_date=start_date,
              end_date=end_date,
              api_key=api_key)


### OECD

In [ ]:
import src.data.oecd_data_retrieval as oecd

oecd.download(start_date=start_date,
              end_date=end_date)

### Merge

In [26]:
import src.data.fred_oecd_data_concatenation as merge
merge.merge()

### Load data

Now, we'll open the downloaded monthly data at a pandas Data Frame:

In [29]:
df = pd.read_csv('data/processed/monthly_data.csv')

# Standardize data handling with pandas Timestamp
df['Date'] = df['Date'].map(lambda x: pd.Timestamp(x))
df.set_index('Date', inplace=True)

# Visualize the requested data
df.head(6)[['USD', 'GBP', 'EUR', 'VIX', 'USDi', 'GBPi', 'EURi']]

,USD,GBP,EUR,VIX,USDi,GBPi,EURi
Date,,,,,,,
2002-04-30,1.6216,2.361860,1.459764,21.91,1.87,4.17319,3.4069
2002-05-31,1.5693,2.295101,1.465569,19.98,1.82,4.15370,3.4671
2002-06-30,1.4920,2.274554,1.470515,25.40,1.81,4.17758,3.4640
2002-07-31,1.4833,2.317656,1.453041,32.03,1.79,4.06054,3.4100
2002-08-31,1.5007,2.322483,1.471586,32.64,1.73,3.98654,3.3519
2002-09-30,1.4758,2.317006,1.457943,39.69,1.76,3.98872,3.3101


## Analysis

### FX rates in CHF

The following plot reproduces the FX rates for the specified countries list:

import src.analysis.performances as plot

## LaTeX

Now, let's go through the compiling process of the LaTeX file into a pdf:

In [31]:
import subprocess

def compile_latex(file_path):
    try:
        # Run pdflatex to compile the LaTeX file
        subprocess.run(['pdflatex', file_path])

        # Additional runs to resolve references (if needed)
        subprocess.run(['pdflatex', file_path])
        subprocess.run(['pdflatex', file_path])

        print("Compilation successful.")
    except subprocess.CalledProcessError as e:
        print(f"Compilation failed. Error code: {e.returncode}")

# Example usage
latex_file_path = 'latex/paper/Currency_risks_for_Swiss_investors.tex'
compile_latex(latex_file_path)
